In [46]:
import cv2
import time
import json
import random

In [200]:
cap = cv2.VideoCapture(0)

In [207]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [208]:
while True:
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(rgb)

    obuff=cv2.resize(rgb, (64,64))
    #cv2.thresh(obuff, )
    #ret,thresh4 = cv2.threshold(obuff,127,255,cv2.THRESH_TOZERO)

    nobuff = bytearray(int(64*64/8))
    cbit = 0
    cpos = 0
    for i in range(64):
        for j in range(64):
            bout = obuff[i][j] > 128
            if (bout):
                nobuff[cbit] |= 1 << cpos
            cpos += 1
            if cpos >= 8:
                cpos = 0
                cbit += 1
    print(nobuff)

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
        off += 64
        #print((lensbuff)

        ln = arduino.write(sbuff)
        print('%s' % ln)
        time.sleep(0.1)
    #print('-'*80)    
    cv2.imshow('frame', obuff)
    time.sleep(2)
    #break


bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00*\x00\x18\x00\x00\x00\x00\x00\x00\x04\x18\x00\x00\x00\x00\x00 \x02\x18\x00\x00\x00\x00\x00\x00\x06\x18\x00\x00\x00\x00\x00\x00\x02\x18\x00\x00\x00\x00\x00\x00Y<\x00\x00\x00\x00\x00\x01\x80~\x00\x00\x00\x00\x00\x07\x80}\x00\x00\x00\x00\x00\x0f\xe0|\x00\x00\x00\x00\x00?\x80N\x00\x00\x00\x00\x00\x7f\x00\x0e\x00\x00\x00\x00\x00\x7f\x03\x0f\x00\x00\x00\x00\x00\xff\x03\r\x00\x00\x00\x00\x00\xff\x0f\r\x00\x00\x00\x00\x00\xff\x17\x1d\x00\x00\x00\x00\x00\xff\xff\x1c\x00\x00\x00\x00\x00\xff\x7f\x1f\x00\x00\x00\x00\x00\xf8\xff?\x00\x00\x00\x00\x00\xe0\xff?\x18\x00\x00\x00\x00\x80\xff\x7f\x00\x00\x00\x00\x00\x01\xfe\x7f\x00\x00\x00\x00\x00\x0

65
65
65
65
65
65
bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\x16\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x00\x1b\x00\x00\x00\x00\x00\x00\x007\x01\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00?\x00\x00\x00\x00\x00\x00\x00\xbc\x00\x00\x00\x00\x00\x00\x00h\x01@\x00\x00\x00\x00\x00\xb0\xcc\xe0\x00\x00\x00\x00\x00\x80\x86\xe0\x00\x00\x00\x00\x00\x80\xba\xe0\x00\x00\x00\x00\x00\x00\x99\xea\x00\x00\x00\x00\x00\x01\x10\xe0\x01\x00\x00\x00\x00\x07\xe0\xee\x03\x00\x00\x00\x00\x1f\xf0\xe4\x03\x00\x00\x00\x00?\xe0\xff\x01\x00\x00\x00\x00\xff\x80{\x03\x00\x00\x00\x00\xff\x013\x00\x00\x00\x00\x00\xff\x03:\x00\x00\x00\x00\x00\xff\x07p\x00\x00\x00\x00\x00\xff\x1ft\x00\x00\x00\x00\x00\xff\x7ft\x00\x00\x00\x00\x00\xff\xff|\x00\x00\x00\x00\x00\xfc\xbf\xf9\x00\x00\x00\x00\x00\xf0\xff\xff@\x00\x00\x00\x00\xe0\xff\xffa\x00\x00\x00\x00\x80\xff\xff\x83\x00\x00\x00\x00\x03\xfe\xff\x03\x00\x00\x00\x00\x1f\xf8\xff\x03\x00\x00\x00\x00\x7f\xe0\xff\x03\x00

65
65
65
65
65
65
bytearray(b'\x0f\x08\x00\x00\x00\x00\x00\x00?\x00\x00\x01\x00\x00\x00\x00\x1f\x00\x80\x01\x00\x00\x00\x00\x7f\x00\x80\x01\x00\x00\x00\x00\xff\x01\xc2\x01\x00\x00\x00\x00\xff\x03\xc0\x01\x00\x00\x00\x00\xff\x01\xc0\x01\x00\x00\x00\x00\xff\x02\xc8\x00\x00\x00\x00\x00\xfe\x07\xe4\x00\x00\x00\x00\x00\xfc\x07\xe8\x00\x00\x00\x00\x00\xf8\x1f\xe8\x01\x00\x00\x00\x00\xf8\x1f\xf8\x01\x00\x00\x00\x00\xf0/\xe8\x01\x00\x00\x00\x00\xe0?\xf0\x01\x00\x00\x00\x00\xc1\x7fp\x01\x00\x00\x00\x00\x83\xffp\x01\x00\x00\x00\x00\x07\xffx\x00\x00\x00\x00\x00\x0f\xffs\x00\x00\x00\x00\x00\x1f\xfe\xe5 \x00\x00\x00\x00?\xfc\xef \x00\x00\x00\x00\x7f\xf8\xf7!\x00\x00\x00\x00\xff\xf0\xffq\x00\x00\x00\x00\xfe\xe1\xffs\x00\x00\x00\x00\xfc\xe3\xffC\x00\x00\x00\x00x\xc7\xff\x03\x00\x00\x00\x00\xf0\x8f\xff\x03\x00\x00\x00\x00\xe1\x0f\xff\x03\x00\x00\x00\x00\xc3?\xfe\x03\x00\x00\x00\x00\x87?\xfc\x03\x00\x00\x00\x00\x0f\x7f\xf8\x03\x00\x00\x00\x00?\xfe\xf1\r\x00\x00\x00\x00~\xfc\xf1\x01\x00\x00\x00\x00\xfc\

65
65
65
65
65
bytearray(b'\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x008\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x01\x00\x00<\x00\x00\x00\x00\x02\x00\x00}\x00\x00\x00\x00\x07\x00\x00\xfc\x00\x00\x00\x00\x13\x00\x00\xfe\x00\x00\x00\x00\x07\x00\x00\x8e\x00\x00\x00\x00\x1f\x00\x00\x0c\x00\x00\x00\x00\x1f\x00\x00\r\x00\x00\x00\x00~\x01\x00\r\x00\x00\x00\x00\xfc\x01\x00\x1c\x00\x00\x00\x00\xf8\x07\x00\x1d\x00\x00\x00\x00\xf0\r\x00\x1f\x00\x00\x00\x00\xc0-\x00\x1c\x00\x00\x00\x00\x81\x13\x00<\x00\x00\x00\x00\x07\x13\x00>\x00\x00\x00\x00\x0f\x84\x01x\x00\x00\x00\x00\x1fx\x00|`\x00\x00\x00\x7fP\x00\xf8\x00\x00\x00\x00\xfc\x00\x05\xf8\x00\x00\x00\x00\xf8\x03\x01\xf8\x01\x00\x00\x00\xe1\x07\x00\xf8\x01\x00\x00\x00\xc3\x0f\x16\xf8\x01\x00\x00\x00\x07?\x00\xf0\x01\x00\x00\x00\x1f~\x00\xf0\x01\x00\x00\x00\x7f\xf8 \xfc\x00\x00\x00\x00\xfe\xf0\x03\xf7\x00\x00\x00\x00\xfc\xc3\x07\xfc\x00\x00\x00\x00\xf1\x87\x1f\xf4\x00\x00\x00\x00\xc7\x1f>\xf4\x00\x00\x00\x00\x0f?|

65
65
65
65
65
bytearray(b'\xe1\xff\x03\x0e\x00\x00\x00\x00\xc1\xff\x07\x0e\x00\x00\x00\x00\x83\xff\x0f\x0f\x00\x00\x00\x00\x07\xff\x1f\x0f\x00\x00\x00\x00\x0f\xfe?\x07\x00\x00\x00\x00\x1f\xfc?\x07\x00\x00\x00\x00?\xf8\x7f\x0f\x00\x10\x00\x00\x7f\xf0\xff\x1f\x00\x00\x00\x00\xff\xe1\xff\x1f\x00\x00\x00\x00\xff\xc3\x7f\x1f\x00 \x00\x00\xfe\x87\x7f\x0b\x00 \x00\x00\xfe\x0f\x7f\x13\x00\x10\x00\x00\xf8\x1f\xfe\x07\x00\x00\x00\x00\xf3?\xfc\x07\x02\x04\x00\x00\xc7\x7f\xf8\x07\x02\x00\x00\x00\x8f\xff\xf0\x0f\x02\x00\x00\x00\x1f\xff\xe3\x1f\x02\x00\x00\x00\x7f\xfc\xc7\x1f\x03!\x00\x00\xff\xf8\x8f\x1f\x03\x10\x00\x00\xff\xf1\x1f\x1f\x83(\x00\x00\xff\xe3?\x1e\x00\x10\x00\x00\xff\x87\x7f>@\x08\x00\x00\xff\x0f\xff? \x04\x00\x00\xff?\xfe?\x00\x00\x00\x00?\x7f\xf8?\x10\x00\x00\x00\x7f\xfe\xf0?\x00\x00\x00\x00\xff\xf9\xe1\xbf\n\x00\x00\x00\xff\xf3\xc7\xff\x01\x00\x00\x00\xff\xe7\x0f\x9f\x04\x00\x00\x00\xff\x9f\x1f\xbe\x02\x00\x00\x00\xff?\x7f|\n\x00\x00\x00\xbf\x7f\xfc|\x08\x00\x08\x02\xff\xfe\xf97\x0

65
65
65
65
65
65


KeyboardInterrupt: 

In [193]:
cv2.destroyAllWindows()

In [186]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

[ 84 101 108 255]


In [176]:
#cap.release()
import random

In [192]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


KeyboardInterrupt: 

In [89]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [90]:
buff=bytearray(65)

In [91]:
arduino.write(b'\x04'+buff)

66

In [107]:
for i in range(65):
    buff[i] = i

In [108]:
for j in range(10):
    for i in range(10):
        buff[0] = i
        ln = arduino.write(buff)
        print('%s' % ln)
        time.sleep(0.1)


65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65
65


In [94]:
while True:
    print(arduino.readline())

b'\x00\x80\x00\x80\x80\x00\x80\x00\x80\x80\x00\x80\x80\x00\x00\x80\x80\x00\x80\x00\x00\x00\x00\x00\x80\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x80\x00\x00\x80\x00\x80\x80\x80\x00\x00\x80\x00\xfe\x80\x00\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x00\x80\x80\x80\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x00\x00\x00\x80\x80\x00\x80\x00\x00\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x00\x80\x00\x80\x80\x80\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x00\x00\x00\x80\x80\x80\x80\x80\x00\x00\x00\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x80\x00\x80\x80\x00\x80\x80\x00\x80\x80\x80\x80\x00\x80\x80\x80\x00\x00\x80\x0065\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65\r\n'
b'65

KeyboardInterrupt: 